### Indian State level analysis like https://www.ft.com/coronavirus-latest

APIs available:

https://api.covid19india.org/data.json

https://api.covid19india.org/state_district_wise.json

https://api.covid19india.org/states_daily.json

https://api.covid19india.org/state_test_data.json

#### Imports

In [1]:
import requests
import json
import plotly.graph_objects as go

#### Get Data

In [2]:
js = json.loads(requests.get("https://api.covid19india.org/states_daily.json").text)['states_daily']

#### Calculate Statewise Daily Deaths with Coronavirus (7-day rolling average)

In [9]:
def get_ravg(status, cutoff):
    def num(s):
        try:
            return int(s)
        except ValueError:
            return 0
    dd = {key: list() for key in js[0].keys() if key not in ['date', 'status', 'tt']}
    for row in js:
        if row['status'] == status:
            for key in dd.keys():
                dd[key].append(row[key])
    def roll_avg(ll):
        if len(ll)>=7:
            return round(sum(ll[-7:]) / 7)
        return round(sum(ll) / len(ll))

    state_flags = dict()
    state_dd_list = dict()
    state_dd_ravg = dict()
    for state in dd.keys():
        state_flags[state] = False
        for state_dd in dd[state]:
            if state_flags[state]:
                state_dd_list[state].append(num(state_dd))
                state_dd_ravg[state].append(roll_avg(state_dd_list[state]))
            elif num(state_dd) >= cutoff:
                state_flags[state] = True
                state_dd_list[state] = [num(state_dd)]
                state_dd_ravg[state] = [num(state_dd)]
    fig = go.Figure()
    annotations = list()
    for key in state_dd_ravg.keys():
        fig.add_trace(go.Scatter(x=list(range(1, (len(state_dd_ravg[key])+1))), y=state_dd_ravg[key],
                             mode='lines',
                             name=key.upper()))
        ann = dict(x=len(state_dd_ravg[key]),
            y=state_dd_ravg[key][-1],
            xref="x",
            yref="y",
            text=key.upper(),
            showarrow=False,
            arrowhead=1,
            ax=0,
            ay=-10)
        annotations.append(dict(ann))
        
        fig.update_layout(showlegend=False, annotations=annotations)

    return fig


#### Plots

In [10]:
fig = get_ravg('Deceased', 3)
fig.update_layout(title='Indian State-wise Daily deaths with coronavirus (as of 17th April, 2020)',
                   xaxis_title='Days since 3 daily deaths first recorded',
                   yaxis_title='Daily Deaths (7-day rolling avg)')

fig.show()

In [11]:
fig = get_ravg('Confirmed', 30)
fig.update_layout(title='Indian State-wise daily confirmed with coronavirus (as of 17th April, 2020)',
                   xaxis_title='Days since 30 daily confirmed first recorded',
                   yaxis_title='Daily confirmed cases (7-day rolling avg)')

fig.show()